In [17]:
from collections import Counter
import sqlite3, json
import pandas as pd

# 1. Connect to the database
conn = sqlite3.connect('./valorant_esports.db')

# 2. Use a SQL query to fetch data directly into a DataFrame
# query = "SELECT * FROM series"
    # Connect to the SQLite database
# conn = get_db()
cursor = conn.cursor()

playerA_id = str(14)
playerB_id = 10612

# SQL query to fetch series_id
query = """
    SELECT series_id
    FROM 'all-players'
    WHERE player_id = ?
    GROUP BY series_id
    ORDER BY COUNT(*) DESC
    LIMIT 1;
"""

cursor.execute(query, (playerA_id,))
row = cursor.fetchone()
# query = "SELECT * FROM 'all-players'"
# all_players_df = pd.read_sql_query(query, conn)

# all_players_df['damageDealtTargets'] = all_players_df['damageDealtTargets'].apply(json.loads)
# all_players_df['damageDealtSources'] = all_players_df['damageDealtSources'].apply(json.loads)
# all_players_df['abilities'] = all_players_df['abilities'].apply(json.loads)
# all_players_df['objectives'] = all_players_df['objectives'].apply(json.loads)

# Get the most occurring series_id
# series_ids = [row[0] for row in rows]
# series_id = Counter(series_ids).most_common(1)[0][0]
# series_ids
# 3. Close the connection
conn.close()
if row:
    series_id = row[0]
    print(series_id)
else:
    print("No matching series_id found")

# all_players_df.info()

2819705


In [1]:
import requests, json
import pandas as pd

url = "https://api-op.grid.gg/live-data-feed/series-state/graphql"
headers = {"x-api-key": "ngT0RNhKcUIK7MR2ffyh4rcBZdxHH0ACAYOLKnNp"}

# Valorant ID - 6
# LOL ID - 3
# gte: "2025-01-01T15:00:07+02:00"
# lte: "2026-01-20T15:00:07+02:00" 

In [18]:
import sqlite3
conn = sqlite3.connect('./valorant_esports.db')

# 2. Use a SQL query to fetch data directly into a DataFrame
# query = "SELECT * FROM series"
query = "SELECT * FROM 'all-teams'"
all_teams_df = pd.read_sql_query(query, conn)
series_ids = all_teams_df['series_id'].unique().tolist()
len(series_ids)

209

In [ ]:
import time

series_query = """query GetLiveValorantSeriesState($id: ID!) {
  seriesState(id: $id) {
    id
    valid
    updatedAt
    format
    started
    finished    
    teams {
      id
      name
      won
      kills
      score
      killAssistsReceived
      killAssistsGiven
      selfkills
      firstKill
      structuresCaptured 
      structuresDestroyed
      weaponKills{
        id
        weaponName
        count
      }
      teamkills
      teamkillAssistsReceived
      deaths      
      
      multikills{
        id
        numberOfKills
        count
      }
      players{
        ...seriesPlayerStateValorant
      }
    }
    games {
      id
      sequenceNumber
      teams {
        id
        name
        won
        side
        score
        loadoutValue
        netWorth
        kills
        players {
           ... gamePlayerStateValorant
        }
      }
    }
  }
}

fragment seriesPlayerStateValorant on SeriesPlayerStateValorant {
        id
        name
          participationStatus
        kills
        killAssistsReceived
        killAssistsGiven
  teamkills
  weaponKills{
    id
    weaponName
    count
  }
        objectives{
          type
          id
        }
          headshots
          teamHeadshots
          structuresCaptured
          structuresDestroyed
          
        }
        
fragment gamePlayerStateValorant on GamePlayerStateValorant {
            id
            name
            participationStatus 
            character{
              id
              name
            } 
              inventory{
    items{
      id
      name
      quantity
      equipped
      stashed
    }
  }
  position {
    x
    y 
  }
  
  objectives{
    id
    type
    completedFirst
    completionCount 
  }
    money
  loadoutValue
  netWorth  
            damageDealtTargets{
              target {
                name
              }
              damageAmount
              occurrenceCount
            }
            damageDealtSources{
              source {
                name
              }
              damageAmount
              occurrenceCount
            }
            
            teamdamageTaken
            teamdamageDealt
            selfdamageTaken
            selfdamageDealt
            damageTaken
            damageDealt
            ultimatePoints
            teamkills
            teamHeadshots
            alive
            maxHealth
            currentArmor
            currentHealth
            statusEffects {
              id
              name
            }
            abilities {
              id
              name
              charges
            }
            unitKills{
              id
              unitName
              count
            }
            
          }
"""

# series_ids = [2819695,2843069, 2843067, 2843063, 2843070, 2819700, 2843071, 2843068, 2843061, 2819704, 2843066,2843060, 2843062, 2819694, 2843064, 2819703, 2819701, 2819698,2843065 , 2819696, 2819699, 2819705]
def fetch_and_save_series(ids):
    with open("./json/all-players-v2.json", "a") as f:  # "a" for append mode
        for sid in ids:
            print(f"Fetching Series: {sid}...")
            
            # 2. Pass the ID into the 'variables' dictionary
            payload = {
                "query": series_query,
                "variables": {"id": sid}
            }
            
            response = requests.post(url, json=payload, headers=headers)
            
            if response.status_code == 200:
                data = response.json()['data']['seriesState']
                # 3. Append to file as a single line (NDJSON style)
                f.write(json.dumps(data) + "\n")
            else:
                print(f"Error on {sid}: {response.status_code}")
            time.sleep(0.5)

fetch_and_save_series(series_ids[:2])

Fetching Series: 2653993...


KeyError: 'seriesState'

## Feature Engineering

In [11]:
import pandas as pd
jsn = pd.read_json('./200_series.json')
jsn = jsn.drop(columns=['valid', 'started', 'finished'])
jsn.info()

<class 'pandas.DataFrame'>
RangeIndex: 209 entries, 0 to 208
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         209 non-null    int64 
 1   updatedAt  209 non-null    str   
 2   format     209 non-null    str   
 3   teams      209 non-null    object
dtypes: int64(1), object(1), str(2)
memory usage: 6.7+ KB


In [ ]:
# # To create a "Vandal Kills" column, "Phantom Kills" column, etc.
# def flatten_weapons(weapon_list):
#     return {item['weaponName']: item['count'] for item in weapon_list}

# # Apply and expand into new columns
# weapon_counts_df = team_table['weaponKills'].apply(flatten_weapons).apply(pd.Series).fillna(0)

In [ ]:
import json 

# NECESSARY_PLAYER_IDS = ['10612', '10636', '1195', '2348', '272','2513','1193','297', '2358', '2340']
jsn = jsn.rename(columns={'id' : 'series_id'} )
df_games = jsn.explode('games')

game_basics = pd.json_normalize(df_games['games'])
game_basics['series_id'] = df_games['series_id'].values
game_basics = game_basics.rename(columns={'sequenceNumber': 'round'})

# 2. Explode Teams (KEEPING series_id and round)
# We select the columns we want to carry down + the 'teams' list
game_teams_exploded = game_basics[['series_id', 'round', 'teams']].explode('teams')

# 3. Normalize Teams to get Team Name and the Players list
team_details = pd.json_normalize(game_teams_exploded['teams'])
# Re-attach the ID columns using .values to ensure alignment
team_details['series_id'] = game_teams_exploded['series_id'].values
team_details['round'] = game_teams_exploded['round'].values
team_details = team_details.rename(columns={'name': 'team_name', 'id': 'team_id' , 'loadoutValue': 'team_loadoutValue', 'netWorth' : 'team_netWorth', 'kills' : 'team_kills'})
print(team_details.info())
# 4. Explode Players (KEEPING IDs and Team Name)
players_exploded = team_details[['series_id', 'round', 'won', 'side' , 'score', 'team_loadoutValue', 'team_netWorth', 'team_kills','team_id', 'team_name', 'players']].explode('players')

# 5. Final Normalization of Player Stats
player_stats_final = pd.json_normalize(players_exploded['players'])

# 6. The Final Merge: Bring IDs and Stats together
final_round_df = pd.concat([
    players_exploded[['series_id', 'round', 'won', 'side' , 'score', 'team_loadoutValue', 'team_netWorth', 'team_kills','team_id', 'team_name',]].reset_index(drop=True),
    player_stats_final.reset_index(drop=True)
], axis=1)

# Extract both 'weaponName' and 'count' as a list of dictionaries
final_round_df['abilities'] = final_round_df['abilities'].apply(
    lambda x: [
        {'abilityName': item['name'], 'charges': item.get('charges', 0)} 
        for item in x
    ] if isinstance(x, list) else []
)

# To use it later:
# df['weaponKills'] = df['weaponKills'].apply(json.loads) # Back to list of objects
# final_round_df = final_round_df.replace({True: 1, False: 0})
final_round_df = final_round_df.rename(columns={'name': 'player_name', 'id': 'player_id'})
final_round_df = final_round_df.drop(columns=['unitKills', 'maxHealth', 'teamHeadshots', 'statusEffects'])

def simplify_damage(damage_list):
    if not isinstance(damage_list, list):
        return []
    
    return [
        {
            "target": item['target']['name'], # Extract just the name string
            "damageAmount": item['damageAmount'],
            "occurrenceCount": item['occurrenceCount']
        }
        for item in damage_list
    ]

def simplify_damage_source(damage_list):
    if not isinstance(damage_list, list):
        return []
    
    return [
        {
            "target": item['source']['name'], 
            "damageAmount": item['damageAmount'],
            "occurrenceCount": item['occurrenceCount']
        }
        for item in damage_list
    ]

# Apply the transformation
final_round_df['damageDealtTargets'] = final_round_df['damageDealtTargets'].apply(simplify_damage)
final_round_df['damageDealtSources'] = final_round_df['damageDealtSources'].apply(simplify_damage_source)

final_round_df['damageDealtTargets'] = final_round_df['damageDealtTargets'].apply(json.dumps)
final_round_df['damageDealtSources'] = final_round_df['damageDealtSources'].apply(json.dumps)
final_round_df['objectives'] = final_round_df['objectives'].apply(json.dumps)
final_round_df['abilities'] = final_round_df['abilities'].apply(json.dumps)
final_round_df['inventory.items'] = final_round_df['inventory.items'].apply(json.dumps)
final_round_df =  final_round_df.drop(columns=['character.name'])
# To use it later:
# df['damageDealtTargets'] = df['damageDealtTargets'].apply(json.loads) # Back to list of objects
final_round_df['won']

<class 'pandas.DataFrame'>
Index: 112 entries, 0 to 21
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   team_id            112 non-null    str   
 1   team_name          112 non-null    str   
 2   won                112 non-null    bool  
 3   side               112 non-null    str   
 4   score              112 non-null    int64 
 5   team_loadoutValue  112 non-null    int64 
 6   team_netWorth      112 non-null    int64 
 7   team_kills         112 non-null    int64 
 8   players            112 non-null    object
 9   series_id          112 non-null    int64 
 10  round              112 non-null    int64 
dtypes: bool(1), int64(6), object(1), str(3)
memory usage: 9.7+ KB
None


0      False
1      False
2      False
3      False
4      False
       ...  
555    False
556    False
557    False
558    False
559    False
Name: won, Length: 560, dtype: bool

In [42]:
final_round_df.info()

<class 'pandas.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 35 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   series_id            560 non-null    int64  
 1   round                560 non-null    int64  
 2   won                  560 non-null    bool   
 3   side                 560 non-null    str    
 4   score                560 non-null    int64  
 5   team_loadoutValue    560 non-null    int64  
 6   team_netWorth        560 non-null    int64  
 7   team_kills           560 non-null    int64  
 8   team_id              560 non-null    str    
 9   team_name            560 non-null    str    
 10  player_id            560 non-null    str    
 11  player_name          560 non-null    str    
 12  participationStatus  560 non-null    str    
 13  objectives           560 non-null    str    
 14  money                560 non-null    int64  
 15  loadoutValue         560 non-null    int64  
 16  n

In [43]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///valorant_esports.db')

final_round_df.to_sql('all-players', con=engine, if_exists='replace')

560

In [ ]:
final_round_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 560 entries, 0 to 559
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   series_id           560 non-null    int64 
 1   round               560 non-null    int64 
 2   team_name           560 non-null    object
 3   team_id             560 non-null    object
 4   player_id           560 non-null    object
 5   player_name         560 non-null    object
 6   damageDealtTargets  560 non-null    object
 7   damageDealtSources  560 non-null    object
 8   teamdamageTaken     560 non-null    int64 
 9   teamdamageDealt     560 non-null    int64 
 10  selfdamageTaken     560 non-null    int64 
 11  selfdamageDealt     560 non-null    int64 
 12  damageTaken         560 non-null    int64 
 13  damageDealt         560 non-null    int64 
 14  ultimatePoints      560 non-null    int64 
 15  teamkills           560 non-null    int64 
 16  alive               560 no

In [ ]:
# 2. Filter the DataFrame
filtered_round_df = final_round_df[final_round_df['player_id'].isin(NECESSARY_PLAYER_IDS)].copy()

# 3. Sort by Series and Round to keep the timeline intact
filtered_round_df = filtered_round_df.sort_values(by=['series_id', 'round'])

# 4. Optional: Verify how many unique target players were actually found
found_players = filtered_round_df['player_id'].nunique()
print(f"Found {found_players} out of {len(NECESSARY_PLAYER_IDS)} target players.")

filtered_round_df

Found 10 out of 10 target players.


,series_id,round,team_name,team_id,player_id,player_name,damageDealtTargets,damageDealtSources,teamdamageTaken,teamdamageDealt,selfdamageTaken,selfdamageDealt,damageTaken,damageDealt,ultimatePoints,teamkills,alive,currentArmor,currentHealth,abilities
340,2819694,1,Cloud9,79,10612,v1c,"[{""target"": ""body"", ""damageAmount"": 705, ""occu...","[{""target"": ""ghost"", ""damageAmount"": 306, ""occ...",0,0,0,0,951,2213,7,0,False,0,0,"[{""abilityName"": ""stim-beacon"", ""charges"": 0},..."
341,2819694,1,Cloud9,79,10636,OXY,"[{""target"": ""body"", ""damageAmount"": 1510, ""occ...","[{""target"": ""paint-shells"", ""damageAmount"": 37...",0,0,0,0,762,2138,4,0,True,50,100,"[{""abilityName"": ""blast-pack"", ""charges"": 0}, ..."
342,2819694,1,Cloud9,79,1195,mitch,"[{""target"": ""body"", ""damageAmount"": 924, ""occu...","[{""target"": ""classic"", ""damageAmount"": 166, ""o...",0,0,0,0,1090,1563,4,0,False,0,0,"[{""abilityName"": ""trailblazer"", ""charges"": 0},..."
343,2819694,1,Cloud9,79,1193,Xeppaa,"[{""target"": ""general"", ""damageAmount"": 10, ""oc...","[{""target"": ""razorvine"", ""damageAmount"": 10, ""...",0,0,0,0,832,1810,3,0,False,0,0,"[{""abilityName"": ""shear"", ""charges"": 0}, {""abi..."
344,2819694,1,Cloud9,79,2348,neT,"[{""target"": ""body"", ""damageAmount"": 473, ""occu...","[{""target"": ""ghost"", ""damageAmount"": 40, ""occu...",0,0,0,0,976,1115,1,0,True,0,9,"[{""abilityName"": ""poison-cloud"", ""charges"": 0}..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
184,2843071,2,NRG,97,2358,mada,"[{""target"": ""body"", ""damageAmount"": 1370, ""occ...","[{""target"": ""classic"", ""damageAmount"": 26, ""oc...",0,0,3,3,2610,2265,1,0,False,0,0,"[{""abilityName"": ""blast-pack"", ""charges"": 1}, ..."
191,2843071,3,NRG,97,2513,brawk,"[{""target"": ""head"", ""damageAmount"": 982, ""occu...","[{""target"": ""sheriff"", ""damageAmount"": 503, ""o...",0,26,0,0,1897,3059,8,0,False,0,0,"[{""abilityName"": ""shock-bolt"", ""charges"": 1}, ..."
192,2843071,3,NRG,97,297,Ethan,"[{""target"": ""head"", ""damageAmount"": 1461, ""occ...","[{""target"": ""classic"", ""damageAmount"": 238, ""o...",0,9,0,0,2278,2454,1,0,False,0,0,"[{""abilityName"": ""frag-ment"", ""charges"": 0}, {..."
193,2843071,3,NRG,97,2340,s0m,"[{""target"": ""body"", ""damageAmount"": 392, ""occu...","[{""target"": ""ghost"", ""damageAmount"": 174, ""occ...",35,0,0,0,2334,1383,7,0,False,0,0,"[{""abilityName"": ""paranoia"", ""charges"": 0}, {""..."


In [ ]:
engine = create_engine('sqlite:///valorant_esports.db')

filtered_round_df.to_sql('players', con=engine, if_exists='replace')

157